Data Flow Between Agents

The system's data flow is coordinated by the Task Planner Agent:

1. Initial Flow: Document → Task Planner → Pre-processor
2. Information Extraction: Pre-processor → Context Bank & Task Planner
3. Knowledge Gathering: Task Planner → Knowledge Agent → Context Bank & Task Planner
4. Compliance Analysis: Task Planner → Compliance Checker (accessing Context Bank)
   - If knowledge is insufficient → Knowledge Agent (with the missing fields)
   - If knowledge is sufficient → Check compliance for each clause
5. Conditional Processing:
   - If contradictions: Compliance Checker → Clause Rewriter → Compliance Checker
   - If compliant: Compliance Checker → Task Planner
6. Summarizing Changes: Task Planner → Post-processor
7. Task Completion: Post-processor → Final Output → User


In [17]:
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
import os
from agents.utils.ollama_client import OllamaClient
from agents.utils.api_client import APIClient
from typing import Any, Dict, Optional

In [18]:
# model = ChatOllama(model="llama3.1:latest")

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")

tavily_api_key = os.getenv("TAVILY_API_KEY")
if not tavily_api_key:
    raise ValueError("TAVILY_API_KEY not found in environment variables. Please set it in your .env file.")

print("[SETUP] Tavily API Key loaded successfully.")

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.1,
    google_api_key=google_api_key
)
print("[SETUP] Initialized ChatGoogleGenerativeAI model with gemini-2.0-flash")

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=google_api_key
)
print("[SETUP] Initialized GoogleGenerativeAIEmbeddings model")

[SETUP] Tavily API Key loaded successfully.
[SETUP] Initialized ChatGoogleGenerativeAI model with gemini-2.0-flash
[SETUP] Initialized GoogleGenerativeAIEmbeddings model


In [19]:
# Initialize the LLM client to be used by tools

def _initialize_llm_client(use_ollama: bool, model_name: str) -> Any:
    """Initialize and return the appropriate LLM client based on settings."""
    try:
        if use_ollama:
            print(f"[SETUP] Initializing Ollama client with model {model_name}")
            return OllamaClient(model_name)
        else:
            print(f"[SETUP] Initializing API client with model {model_name}")
            return APIClient(model_name)
    except Exception as e:
        print(f"[ERROR] Failed to initialize LLM client: {str(e)}")
        return None

## Context Bank Initialization

Initialize a shared context bank instance that will be used by all agents to store and retrieve document information.


In [20]:
from context_bank import ContextBank

# Initialize a single shared context bank instance
context_bank = ContextBank()

# This context_bank will be passed to all agents that need to store or retrieve information

In [21]:
planner_agent_tools = [
    create_handoff_tool(agent_name="Pre Processor Agent", description="Transfer when pre-processing is needed, it helps to format and clean the input data."),
    create_handoff_tool(agent_name="Knowledge Agent", description="Transfer when knowledge is needed, it helps to retrieve knowledge from the web using websearcher."),
    create_handoff_tool(agent_name="Compliance Checker Agent", description="Transfer when compliance checking is needed, it helps to check legal documents for compliance with regulations."),
    create_handoff_tool(agent_name="Post Processor Agent", description="Transfer when post processing is needed, it helps to format and finalize the output."),
]

planner_agent_node = create_react_agent(
    model,
    planner_agent_tools,
    prompt="""
        You are a Task Planner Agent responsible for coordinating a multi-agent system to analyze legal documents for discrepancies and compliance. Your job is to plan and delegate tasks to specialized agents using the relevant handoff tools and track task completion.

        INPUTS:
        Problem to solve: use the user prompt
        Analyze a legal document for discrepancies and compliance issues.

        PLANNED TASKS:

        * Preprocess document
        * Extract and classify clauses
        * Retrieve relevant legal compliance knowledge from the web
        * Check clause compliance and legal discrepancies
        * Summarize issues and finalize output

        AVAILABLE AGENTS:
        * Pre Processor Agent: Responsible for pre-processing the document, extracting clauses, and classifying them. Processes the input legal document, adds all the relevant information to the context bank and returns status.
        * Knowledge Agent: Responsible for retrieving relevant legal compliance knowledge from the web. Fetches information from the web, adds all the relevant knowledge to a vector DB and returns status.
        * Compliance Checker Agent: Responsible for checking the compliance of clauses with legal regulations. Performs the compliance check, returns a list of non-compliant clauses and their details. Also returns status.
        * Post Processor Agent: Responsible for summarizing issues and finalizing the output. Formats the final output and returns a summary of the compliance check results. Also returns status.

        ACTION: 
        [IMPORTANT] Status Check - Check status of Preprocessor, Compliance Checker, and Post-Processor agents

        If Preprocessor status is not complete, trigger Preprocessor Agent.
        Once preprocessing is complete, trigger Knowledge Agent.
        After Knowledge Agent retrieves relevant knowledge, trigger Compliance Checker Agent.
        Upon clause revision, Post Processor Agent is triggered for final output and summary.

        Rationale:
        [EXTREMELY CRITICAL] Each agent’s task is sequentially dependent, ensure no step is skipped in the workflow. Status check ensures no redundant computation and completion of workflow.
    """,
    name="Planner Agent"
)

In [22]:
import PyPDF2
import re
import spacy
import uuid
import spacy
import json  # Added for pretty printing

# Download the spaCy model if it's not already installed
!python -m spacy download en_core_web_sm

# Load once to avoid redundant loading
spacy_ner = spacy.load("en_core_web_sm")
print("[SETUP] Loaded spaCy NER model en_core_web_sm")

llm_client = _initialize_llm_client(use_ollama=False, model_name="gemini-2.0-flash")

def preprocess_document_tool_implementation(file_path: str, context_bank, system_prompt) -> dict:
    """
    Consolidated preprocessing tool to be used as a callable function in a multi-agent system.
    Extracts text, title, named entities, and clause classifications from a PDF document.
    """
    print("\n" + "="*80)
    print(f"[PREPROCESS] Starting document preprocessing for: {file_path}")
    print(f"[PREPROCESS] Context Bank state at start: {json.dumps(context_bank.get_all(), indent=2)}")
    print("="*80 + "\n")

    # Step 1: Extract text from PDF
    print("[PREPROCESS] Step 1: Extracting text from PDF...")
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = "".join(page.extract_text() for page in reader.pages)
    print(f"[PREPROCESS] Extracted {len(text)} characters of text")

    # Step 2: Title Extraction (from first 10 lines)
    # print("[PREPROCESS] Step 2: Extracting title...")
    # def extract_title(text: str) -> str:
    #     lines = text.split("\n")
    #     candidates = []
    #     for i, line in enumerate(lines[:10]):
    #         clean_line = line.strip()
    #         if not clean_line or len(clean_line) < 5:
    #             continue
    #         score = 0
    #         if re.match(r"^(CONTRACT|AGREEMENT|PETITION|NOTICE|ORDER|BILL|ACT|STATUTE)\b", clean_line, re.IGNORECASE):
    #             score += 5
    #         if re.match(r"^[A-Z\s\-]{5,}$", clean_line):
    #             score += 2
    #         if "**" in clean_line or clean_line.center(80) == clean_line:
    #             score += 1
    #         candidates.append((clean_line, score))
    #     candidates.sort(key=lambda x: x[1], reverse=True)
    #     return candidates[0][0] if candidates else "Unknown Title"

    print("[PREPROCESS] Step 2: Extracting title...")
    title_query = "what is the title of this document?" + text[:1000]
    title = llm_client.query(title_query)
    print(f"[PREPROCESS] Extracted title: {title}")

    # Step 3: Named Entity Recognition
    print("[PREPROCESS] Step 3: Performing Named Entity Recognition...")
    doc = spacy_ner(text)
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    print(f"[PREPROCESS] Extracted {len(entities)} named entities")
    print(f"[PREPROCESS] Sample entities (first 5): {entities[:5]}")

    # Step 4: Document + Clause Classification via external LLM system
    print("[PREPROCESS] Step 4: Classifying document and clauses...")
    llm_output = llm_client.query(text, system_prompt)

    if isinstance(llm_output, str):
        # strip any enclosing backticks (```), whitespace, etc.
        llm_output = llm_output.strip().strip("```json").strip("`")
        # remove literal "\\n" sequences that came escaped
        llm_output = llm_output.replace("\\n", "")
        try:
            llm_output = json.loads(llm_output)
            print(f"[PREPROCESS] Successfully parsed LLM output as JSON")
        except json.JSONDecodeError as e:
            print(f"[ERROR] Failed to parse LLM output as JSON: {e}")
            print(f"[ERROR] Raw output: {llm_output[:500]}...")
            raise RuntimeError("Failed to parse LLM output as JSON:\n" + llm_output)

    document_class = llm_output.get("CLASS", "")
    clause_classes = llm_output.get("CLAUSES", [])
    
    print(f"[PREPROCESS] Document class: {document_class}")
    print(f"[PREPROCESS] Extracted {len(clause_classes)} clauses")

    # Step 5: Soring in Context Bank
    print("[PREPROCESS] Step 5: Storing in Context Bank...")
    context_bank.add_document(text, {
        "title": title,
        "document_type": document_class,
        "source_file": file_path
    })
    context_bank.add_entities(entities)
    context_bank.add_clauses(clause_classes)
    print("[PREPROCESS] Data successfully stored in Context Bank")
    
    print("\n" + "="*80)
    print(f"[PREPROCESS] Context Bank state after processing: {json.dumps(context_bank.get_clauses(), indent=2)}")
    print("="*80 + "\n")

    # Final structured output
    return {
        "Document Title": title,
        "Document Class": document_class,
    }

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
[SETUP] Loaded spaCy NER model en_core_web_sm
[SETUP] Initializing API client with model gemini-2.0-flash


In [23]:
@tool
def preprocess_document_tool(file_path: str) -> dict:
    """
    Creates a tool function for preprocessing legal documents.
    
    Args:
        file_path (str): Path to the legal document PDF file.
        
    Returns:
        dict: A dictionary containing the extracted information.
    """

    SYSTEM_PROMPT = """
        You are a Pre-processor Agent, a specialized component in the Legal Document Analysis Framework responsible for extracting critical information from legal documents and storing it in the Context Bank. Your work forms the foundation for all subsequent analysis by other agents in the system.

        Core Responsibilities:
        Your sole task is to extract and structure information from legal documents, including:
        - Classifying the document type and purpose
        - Extracting important clauses with their classifications
        - Storing all extracted information in a structured format accessible to other agents
        - Provide your output as strict JSON
        Input:
        Legal Contract Document PDF.

        Output Format:
        {
        "CLASS": "Document type classification (e.g., Legal Agreement - Employment Contract)",
        "CLAUSES": [
            {"Text": "Section 3.1: The term of this agreement shall be...", "Category": "Term Clause"},
            {"Text": "Section 7.2: All disputes shall be resolved by...", "Category": "Dispute Resolution"},
            {"Text": "Section 9.5: This agreement shall be governed by...", "Category": "Governing Law"}
        ]
        }

        [EXTREMELY CRITICAL] Ensure that the output is strictly in JSON format. Do not include any additional text or explanations. The output must be parsable as JSON.
    """
        
        # Call the implementation with the shared context bank
    return preprocess_document_tool_implementation(file_path, context_bank, SYSTEM_PROMPT)

In [24]:
pre_processor_agent_tools = [
    create_handoff_tool(agent_name="Planner Agent", description="Transfer when pre-processing is completed, it helps to plan the next steps in the workflow and delegate tasks."),
    preprocess_document_tool
]

pre_processor_agent_node = create_react_agent(
    model,
    pre_processor_agent_tools,
    prompt="""
         You are the Pre-processor Agent in a Legal Document Analysis Framework. Your sole function is to extract critical information from legal document PDFs and structure it for other agents.

         Core Task:
         Using the provided tool, preprocess_document_tool, process an input legal document PDF to:
         1.   Extract Full Text:  Get the complete text content.
         2.   Classify Document:  Determine the document type (e.g., NDA, Lease, Employment Agreement).
         3.   Identify Named Entities (NER):  Extract key entities (Parties, Laws, Dates, Jurisdictions, Monetary Values, etc.).
         4.   Extract Key Clauses:  Isolate and classify significant clauses (e.g., Term, Governing Law, Confidentiality).
         5.   Store Data:  Structure all extracted information (Text, Class, NER list, Clauses list) as a JSON object in the Context Bank.

         Input:  Legal Contract Document PDF.
         Output:  JSON object with "TEXT", "CLASS", "NER", and "CLAUSES" as keys.

         Guidelines: 
         * Be accurate and comprehensive.
         * Preserve original context, especially for clauses.
         * Focus on legally significant information and obligations.
         * Note any low-confidence classifications.
         * [CRITICAL STEP] Return to the Planner Agent to update that the pre-processing is completed.
      """,
    name="Pre Processor Agent"
)

In [ ]:
import requests
import uuid
import traceback
from typing import List, Dict
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from cleantext import clean
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, PointStruct, Distance
# from langchain_ollama import OllamaEmbeddings
from langchain_core.tools import tool
from tavily import TavilyClient

# Initialize global components once
_qdrant_url = "http://localhost:6333"
_qdrant_collection = "web_content"
_qdrant_client = QdrantClient(
    url=_qdrant_url, 
    prefer_grpc=False
)

_num_results = 3
_ddgs = DDGS()

# Use the GoogleGenerativeAIEmbeddings model instead of OllamaEmbeddings
_embeddings_model = embeddings

# Optional: create collection if it doesn't exist
# Get the embedding dimension from the model by embedding a test string
test_embedding = _embeddings_model.embed_query("test")
vector_size = len(test_embedding)
print(f"[SETUP] Generated test embedding with dimension: {vector_size}")

try:
    print(f"[SETUP] Creating Qdrant collection with vector size: {vector_size}")
    if not _qdrant_client.collection_exists(_qdrant_collection):
        _qdrant_client.create_collection(
            collection_name=_qdrant_collection,
            vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
        )
    else:
        print(f"[SETUP] Collection '{_qdrant_collection}' already exists; skipping creation.")

    print(f"[SETUP] Successfully created Qdrant collection '{_qdrant_collection}'")

except Exception as e:
    print(f"[ERROR] Failed to create Qdrant collection. Encountered an exception.")
    print(f"[ERROR] {type(e).__name__}: {e}")
    traceback.print_exc()
        



@tool
def retrieve_web_knowledge_tool(query: str) -> List[Dict]:
    """
    Searches the web for a legal/policy topic,
    scrapes and cleans page content, generates embeddings with Google Gen AI,
    stores them in Qdrant, and retrieves top relevant results.
    Returns a list of summarized search results.
    """

    # Step 1: Tavily search
    print(f"[KNOWLEDGE] Searching Tavily for query: {query}")
    # results = list(_ddgs.text(query, max_results=10))

    tavily_client = TavilyClient(tavily_api_key)
    results = tavily_client.search(query, max_results=10)

    # print(f"[INFO] Tavily results: {results}")

    url_results = results.get("results", [])
    
    print(f"[KNOWLEDGE] Found {len(url_results)} results from Tavily search")


    # Step 2: Scrape + clean + embed + store
    points_to_upsert = []
    successful_urls = 0
    for i, result in enumerate(url_results):
        try:
            url = result["url"]
            title = result["title"]
            print(f"[KNOWLEDGE] Processing result {i+1}/{len(url_results)}: {title}")

            response = requests.get(url, timeout=10)
            response.raise_for_status() # Raise an exception for bad status codes
            soup = BeautifulSoup(response.content, "html.parser")
            paragraphs = soup.find_all("p")
            content = " ".join(p.get_text() for p in paragraphs)
            content = " ".join(content.split()) # Remove extra whitespace
            cleaned_content = clean(
                content,
                fix_unicode=True,
                to_ascii=True,
                lower=True,
                no_line_breaks=True,
                lang="en"
            )

            if not cleaned_content: # Skip if content is empty after cleaning
                print(f"[KNOWLEDGE] No content found or extracted for URL {url}")
                continue

            # Generate embeddings using Google Gen AI embeddings model
            generated_embeddings = _embeddings_model.embed_query(text=cleaned_content)
            point_id = str(uuid.uuid5(uuid.NAMESPACE_URL, url))

            points_to_upsert.append(
                PointStruct( # Use PointStruct for clarity
                    id=point_id,
                    vector=generated_embeddings,
                    payload={
                        "title": title,
                        "content": cleaned_content,
                        "url": url
                    }
                )
            )
            successful_urls += 1
            print(f"[KNOWLEDGE] Successfully processed URL: {url}")
        except requests.exceptions.RequestException as e:
            print(f"[ERROR] Failed to fetch URL {url}: {e}")
        except Exception as e:
            print(f"[ERROR] Failed to process URL {url}: {e}") # Catch other potential errors

    # Upsert points in batch if any were successfully processed
    if points_to_upsert:
        print(f"[KNOWLEDGE] Upserting {len(points_to_upsert)} documents to Qdrant collection")
        try:
            _qdrant_client.upsert(
                collection_name=_qdrant_collection,
                points=points_to_upsert,
                wait=True # Optional: wait for operation to complete
            )
            print(f"[KNOWLEDGE] Successfully upserted {len(points_to_upsert)} documents to Qdrant")
        except Exception as e:
            print(f"[ERROR] Failed to upsert to Qdrant: {e}")
    else:
        print(f"[KNOWLEDGE] No documents to upsert to Qdrant")
        
    print(f"[KNOWLEDGE] Retrieved and processed {successful_urls} out of {len(url_results)} URLs")
    return [{"status": "completed", "total_urls": len(url_results)}]
    # try:
    #     query_vec = _embeddings_model.embed_query(text=query)
    #     search_results = _qdrant_client.search(
    #         collection_name=_qdrant_collection,
    #         query_vector=query_vec,
    #         with_payload=True,
    #         limit=_num_results
    #     ) # search returns Hit objects directly

    #     return [
    #         {
    #             "title": result.payload["title"],
    #             "content": result.payload["content"][:400] + "...",
    #             "url": result.payload["url"],
    #             "score": result.score
    #         }
    #         for result in search_results
    #     ]
    # except Exception as e:
    #     print(f"[ERROR] Failed to search Qdrant: {e}")
    #     return [] # Return empty list on search failure

2025-04-27 16:49:22,339 - httpx - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-04-27 16:49:22,838 - httpx - INFO - HTTP Request: GET http://localhost:6333/collections/web_content/exists "HTTP/1.1 200 OK"


[SETUP] Generated test embedding with dimension: 768
[SETUP] Creating Qdrant collection with vector size: 768
[SETUP] Collection 'web_content' already exists; skipping creation.
[SETUP] Successfully created new Qdrant collection 'web_content'


In [26]:
# TODO : Add the other tools for Knowledge Agent - Web searcher, Context Bank setter and getter

knowledge_agent_tools = [
    create_handoff_tool(agent_name="Planner Agent", description="Transfer to Planner Agent when knowledge has been retrieved and pass a summary back to it."),
    retrieve_web_knowledge_tool
]

knowledge_agent_node = create_react_agent(
    model,
    knowledge_agent_tools,
    prompt="""
        You are a Knowledge Retrieval Agent tasked with extracting accurate, up-to-date legal information from official U.S. government sources.

        Use the retrieve_web_knowledge_tool to fetch up-to-date legal data from trusted government sites and perform the following functions:
        * Retrieve relevant statutes, regulations, and policies based on a given topic.
        * Ensure content is current, authoritative, and clearly summarized.
        * Avoid non-official, outdated, or speculative sources.


        Search Query Format:
        site:[gov source] "[TOPIC]" AND "[FOCUS]" AND ("[KEYWORD1]" OR "[KEYWORD2]") after:[YEAR]

        Sources: congress.gov, govinfo.gov, law.cornell.edu, federalregister.gov, ecfr.gov, justice.gov, whitehouse.gov

        Output Format: Return a simple sentence with the status of the knowledge retrieval.

        Must have title, source and description.

        Key Provisions:
        [Section] — [Impact/Explanation]
        [Section] — [Impact/Detail]

        Guidelines:
        * Use only listed government sources.
        * Do not fabricate or paraphrase inaccurately.
        * If no reliable info is found, say so.
        * [CRITICAL STEP] Return to the Planner Agent to update that the knowledge retrieval is completed.
    """,
    name="Knowledge Agent",
)

In [27]:
from agents.compliance_checker import check_legal_compliance
from context_bank import ContextBank
from langgraph.prebuilt import ToolNode
from agents.utils.websearcher import WebContentRetriever

# Create an instance of WebContentRetriever to use for querying the vector database
def get_knowledge_from_vector_db(query, jurisdiction, document_type: str) -> List[Dict]:
    """
    Retrieves legal knowledge from the vector database based on the query,
    jurisdiction, and knowledge type.
    
    Args:
        query: The search query string
        jurisdiction: The legal jurisdiction (default: "US")
        
    Returns:
        List of relevant knowledge items with title, content, URL, and relevance score
    """
    
    # Refine the query with jurisdiction and knowledge type for better results
    refined_query = f"{query} jurisdiction:{jurisdiction} document_type:{document_type}"
    
    # TODO : Initialize the WebContentRetriever with the appropriate parameters
    try:
        # Initialize the WebContentRetriever with the appropriate collection
        print("[KNOWLEDGE RETRIEVAL] Initializing WebContentRetriever...")
        retriever = WebContentRetriever(
            qdrant_url="http://localhost:6333",
            num_results=5
        )
        
        # Query the vector database
        print("[KNOWLEDGE RETRIEVAL] Querying vector database...")
        search_results = retriever.search_in_qdrant(refined_query)
        print(f"[KNOWLEDGE RETRIEVAL] Retrieved {len(search_results)} search results")
        
        # Return the search results in a structured format
        results = [
            {
                "title": result["title"],
                "content": result["content"],
                "relevance": result["score"],
                "jurisdiction": jurisdiction,
            }
            for result in search_results
        ]
        
        print("\n" + "="*80)
        print(f"[KNOWLEDGE RETRIEVAL] Completed with {len(results)} knowledge items")
        for i, result in enumerate(results):
            print(f"[KNOWLEDGE RETRIEVAL] Result {i+1}: {result['title']} (Score: {result['relevance']})")
        print("="*80 + "\n")
        
        return results
    except Exception as e:
        print(f"[ERROR] Failed to retrieve knowledge from vector DB: {e}")
        # Return empty list on search failure
        return []

# Create a tool that the ComplianceCheckerAgent can use
@tool
def compliance_check_tool(clauses: List[str]) -> List[Dict]:
    """
    Tool for checking legal document clauses for compliance issues.
    
    Args:
        clauses: List of clause dictionaries, each containing 'id' and 'text' keys
        document_metadata: Document metadata (jurisdiction, document_type, etc.)
        
    Returns:
        List of non-compliant clauses with detailed analysis
    """
    
    print("\n" + "="*80)
    print(f"[COMPLIANCE CHECKER] Starting compliance check for {len(clauses)} clauses")
    print(f"[COMPLIANCE CHECKER] Context Bank state at start: {json.dumps(context_bank.get_all(), indent=2)}")
    print("="*80 + "\n")

    query = "Retrieve all relevant legal knowledge for compliance checking."

    jurisdiction = context_bank.get_jurisdiction()
  
    doc_meta_from_bank = context_bank.get_document()
    # Extract the document_type from the retrieved metadata
    # Provide a default value if the key is missing
    document_type = doc_meta_from_bank.get("document_type", "Unknown Document Type") 
    
    print(f"[COMPLIANCE CHECKER] Using jurisdiction: {jurisdiction}")
    print(f"[COMPLIANCE CHECKER] Using document type: {document_type}")

    # Create a knowledge retrieval adapter that mimics a knowledge agent
    # but actually uses the vector DB directly
    print("[COMPLIANCE CHECKER] Retrieving knowledge from vector DB...")
    knowledge_data = get_knowledge_from_vector_db(query, jurisdiction, document_type)
    print(f"[COMPLIANCE CHECKER] Retrieved {len(knowledge_data)} knowledge items")
    
    print("[COMPLIANCE CHECKER] Checking legal compliance...")
    results = check_legal_compliance(
        clauses=clauses,
        document_metadata=doc_meta_from_bank,
        context_bank=context_bank,
        knowledge_from_vector_db=knowledge_data,
        use_ollama=False,
        # model_name="llama3.1:latest",
        model_name="gemini-2.0-flash",
        min_confidence=0.75
    )
    
    print(f"[COMPLIANCE CHECKER] Compliance check completed with {len(results)} results")
    print("\n" + "="*80)
    print(f"[COMPLIANCE CHECKER] Context Bank state after check: {json.dumps(context_bank.get_all(), indent=2)}")
    print("="*80 + "\n")
    
    return results

In [28]:
# TODO : Add the other tools for Compliance Checker Agent - Clause Compliance Checker (which contains the Statutory Validator, Precedent Analyzer, Contractual Consistency Engine, Hypergraph Analyzer, Confidence Scorer), Context Bank getter

compliance_checker_agent_tools = [
    create_handoff_tool(agent_name="Knowledge Agent", description="Transfer to Knowledge Agent if more knowledge is needed, it helps to retrieve knowledge from the web using websearcher."),
    create_handoff_tool(agent_name="Planner Agent", description="Transfer to Planner Agent when compliance checking is completed and all clauses are found to be compliant, it helps to plan the next steps in the workflow and delegate tasks."),
    compliance_check_tool
]

compliance_checker_agent_node = create_react_agent(
    model,
    compliance_checker_agent_tools,
    prompt="""
      You are the Compliance Checker Agent, responsible for analyzing a list of extracted legal clauses to identify contradictions, ensure statutory compliance, and assess contractual consistency under U.S. law.

      Use the compliance_check_tool to perform the following primary functions:
      Detect compliance issues: statutory, precedent-based, internal.
      Validate compliance with federal, state, and city laws
      Ensure internal consistency across clauses
      Identify legal risks and their implications
      Provide structured legal reasoning and confidence scores

      Output:
      1. Contradiction Report
      {
        "has_contradiction": true|false,
        "contradiction_type": "statutory|precedent|internal",
        "severity": "high|medium|low",
        "description": "...",
        "source_clause": { "id": "...", "text": "..." },
        "reference": { "type": "...", "id": "...", "text": "..." }
      }
      2. Reasoning & Analysis
      {
        "analysis_steps": ["Step 1...", "Step 2...", "Step 3..."],
        "confidence_score": 0.0–1.0,
        "supporting_references": [{ "type": "statute", "id": "...", "relevance": "..." }]
      }
      3. Legal Implications
      {
        "implications": [
          {
            "description": "...",
            "severity": "high|medium|low",
            "affected_parties": ["..."],
            "risk_areas": ["..."]
          }
        ]
      }


      [CRITICAL STEP] Decision Flow:
      If information is deemed insufficient, call the Knowledge Agent to retrieve information in the missing field
      If compliance check is complete, return to the Planner Agent for post processing where the whole process is summarized

      Guidelines:
      * Use only validated legal sources
      * No fabrication or assumptions
      * Flag unclear issues and recommend human review when needed
      * Consider jurisdictional scope and maintain objectivity
      * [CRITICAL STEP] Return to the Planner Agent to update that the compliance check is completed.
    """,
    name="Compliance Checker Agent",
)

In [29]:
# # TODO : Add the other tools for Clause Rewriter Agent -

# clause_rewriter_agent_tools = [
#     create_handoff_tool(agent_name="Compliance Checker Agent", description="Transfer to Compliance Checker Agent after a non-compliant clause has been rewritten, it helps to check the compliance of the rewritten clause."),
# ]

# clause_rewriter_agent_node = create_react_agent(
#     model,
#     clause_rewriter_agent_tools,
#     prompt="""
#     You are the Clause Rewriter Agent, tasked with revising legal clauses flagged as non-compliant, contradictory, or unclear by the Compliance Checker Agent. Your goal is to ensure legal compliance while preserving the original intent.

# Responsibilities:
# Rewrite clauses to resolve statutory, precedent-based, or internal contradictions
# Ensure clarity, enforceability, and alignment with U.S. law
# Maintain intent and context of original clause
# Signal if more legal context is required (route to Knowledge Agent)

# Input Format:
# {
#   "original_clause": {
#     "id": "clause_id",
#     "text": "original text"
#   },
#   "issue": {
#     "description": "reason for non-compliance",
#     "contradiction_type": "statutory|precedent|internal",
#     "reference": {
#       "type": "statute|precedent|clause",
#       "text": "reference text",
#       "source_link": "optional"
#     }
#   },
#   "context_info": {
#     "document_title": "Title",
#     "named_entities": [...],
#     "document_class": "e.g., NDA, Lease"
#   }
# }
# Output Format:
# {
#   "clause_id": "clause_id",
#   "rewritten_clause": "Compliant version of the clause",
#   "justification": "How it resolves the issue and aligns with legal standards"
# }
# Guidelines:
# Be concise, precise, and legally sound
# Do not fabricate or generalize
# Flag insufficient context when needed

#     """,
#     name="Clause Rewriter Agent",
# )

In [30]:
# TODO : Add the other tools for Post-Processor Agent - Process Summarizer, Context Bank getter

post_processor_agent_tools = [
    create_handoff_tool(agent_name="Planner Agent", description="Transfer to Planner Agent when post-processing is completed, it helps to plan the next steps in the workflow and delegate tasks."),
]

post_processor_agent_node = create_react_agent(
    model,
    post_processor_agent_tools,
    prompt="""
        You are the Post-processor Agent, responsible for generating final, human-readable outputs after a legal document passes all compliance checks.

        Input:
        Context Bank (document metadata, clause info)
        Compliance Checker outputs (reasoning, implications)
        History of tasks done
        Tools: Summarizer
        Outputs (via Process Summarizer):
        Contract Summary – Overview of the document
        Changes – Highlighted clause modifications
        Risks Averted – Legal issues resolved
        References – Cited statutes and precedents

        Guidelines:
        * Be clear, concise, and legally accurate
        * Avoid jargon or speculation
        * Tailor for legal and business audiences
        * [CRITICAL STEP] Return the summary to the Planner Agent and update that the post processing is completed.
""",
    name="Post Processor Agent",
)

In [31]:
checkpointer = InMemorySaver()
workflow = create_swarm(
    [planner_agent_node, pre_processor_agent_node, knowledge_agent_node, compliance_checker_agent_node, post_processor_agent_node],
    default_active_agent="Planner Agent"
)
app = workflow.compile(checkpointer=checkpointer)

print("\n" + "="*80)
print("[WORKFLOW] Multi-agent swarm initialized with the following agents:")
print("  - Planner Agent")
print("  - Pre Processor Agent")
print("  - Knowledge Agent")
print("  - Compliance Checker Agent")
print("  - Post Processor Agent")
print("[WORKFLOW] Default active agent: Planner Agent")
print("="*80 + "\n")


[WORKFLOW] Multi-agent swarm initialized with the following agents:
  - Planner Agent
  - Pre Processor Agent
  - Knowledge Agent
  - Compliance Checker Agent
  - Post Processor Agent
[WORKFLOW] Default active agent: Planner Agent



In [32]:
config = {"configurable": {"thread_id": "1"}}
turn_1 = app.invoke(
    {"messages": 
        [{
            "role": "user", 
            "content": """You are given a file path the document which you must preprocess to extract clauses. Once the clauses are extracted, fetch all relevant knowledge related to it. Based on the collected knowledge, you should check for compliance of these clauses. Explain the non-compliant clauses, suggest changes and summarize the results for the User. FILE PATH OF DOCUMENT: \"./Original and Modified/modified_UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf\" ",
            """
        }]
    },
    config,
)
print(turn_1)



[PREPROCESS] Starting document preprocessing for: ./Original and Modified/modified_UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf
[PREPROCESS] Context Bank state at start: {
  "document": null,
  "entities": [],
  "clauses": [],
  "laws": {},
  "jurisdiction": null,
  "clause_compliance_results": {},
  "document_analysis": null,
  "non_compliant_clauses": []
}

[PREPROCESS] Step 1: Extracting text from PDF...
[PREPROCESS] Extracted 167916 characters of text
[PREPROCESS] Step 2: Extracting title...
[PREPROCESS] Extracted title: Based on the text provided, the title of the document is:

**NETWORK 1 FINANCIAL CORPORATION AFFILIATE OFFICE AGREEMENT**
[PREPROCESS] Step 3: Performing Named Entity Recognition...
[PREPROCESS] Extracted 3165 named entities
[PREPROCESS] Sample entities (first 5): [('10.11', 'CARDINAL'), ('NETWORK 1 FINANCIAL CORPORATION', 'WORK_OF_ART'), ('AGREEMENT', 'GPE'), ('NETWORK 1 FINANCIAL, INC.', 'ORG'), ('NETWORK 1', 'WORK_OF_ART')]
[PREPROC

2025-04-27 16:49:39,535 - bs4.dammit - WARNING - Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[KNOWLEDGE] No content found or extracted for URL https://www.govinfo.gov/content/pkg/FR-2021-02-12/pdf/2021-01499.pdf
[KNOWLEDGE] Processing result 2/10: Rules and Regulations


2025-04-27 16:49:40,367 - bs4.dammit - WARNING - Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[KNOWLEDGE] No content found or extracted for URL https://www.govinfo.gov/content/pkg/FR-2021-02-23/pdf/2020-28473.pdf
[KNOWLEDGE] Processing result 3/10: Rules and Regulations


...

[KNOWLEDGE] No content found or extracted for URL https://www.govinfo.gov/content/pkg/FR-2024-01-18/pdf/2023-28629.pdf
[KNOWLEDGE] Processing result 9/10: Statement of Policy on Bank Merger Transactions


2025-04-27 16:51:10,226 - bs4.dammit - WARNING - Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[KNOWLEDGE] No content found or extracted for URL https://www.govinfo.gov/content/pkg/FR-2024-09-27/pdf/2024-22189.pdf
[KNOWLEDGE] Processing result 10/10: H. R. 1048


2025-04-27 16:51:11,080 - bs4.dammit - WARNING - Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[KNOWLEDGE] No content found or extracted for URL https://www.govinfo.gov/content/pkg/BILLS-119hr1048eh/pdf/BILLS-119hr1048eh.pdf
[KNOWLEDGE] No documents to upsert to Qdrant
[KNOWLEDGE] Retrieved and processed 0 out of 10 URLs
{'messages': [HumanMessage(content='You are given a file path the document which you must preprocess to extract clauses. Once the clauses are extracted, fetch all relevant knowledge related to it. Based on the collected knowledge, you should check for compliance of these clauses. Explain the non-compliant clauses, suggest changes and summarize the results for the User. FILE PATH OF DOCUMENT: "./Original and Modified/modified_UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf" ",\n            ', additional_kwargs={}, response_metadata={}, id='98c3fc73-97f8-417d-ae54-447a83da5465'), AIMessage(content='Okay, I understand the problem. I need to analyze the legal document at "./Original and Modified/modified_UsioInc_20040428_SB-2_EX-10.11_17239

================================================================================
[PREPROCESS] Starting document preprocessing for: ./Original and Modified/modified_UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf
[PREPROCESS] Context Bank state at start: {
"document": null,
"entities": [],
"clauses": [],
"laws": {},
"jurisdiction": null,
"clause_compliance_results": {},
"document_analysis": null,
"non_compliant_clauses": []
}
================================================================================

[PREPROCESS] Step 1: Extracting text from PDF...
[PREPROCESS] Extracted 167916 characters of text
[PREPROCESS] Extracted title: Based on the provided text, the title of the document is:

**NETWORK 1 FINANCIAL CORPORATION AFFILIATE OFFICE AGREEMENT**
[PREPROCESS] Step 3: Performing Named Entity Recognition...
[PREPROCESS] Extracted 3165 named entities
[PREPROCESS] Step 4: Classifying document and clauses...
[PREPROCESS] Document class: Legal Agreement - Affiliate Office Agreement
[PREPROCESS] Extracted 6 clauses
[PREPROCESS] Step 5: Storing in Context Bank...
[PREPROCESS] Data successfully stored in Context Bank

================================================================================
[PREPROCESS] Context Bank state after processing: [
{
"Text": "THIS AGREEMENT is entered into by and between NETWORK 1 FINANCIAL, INC. (\"NETWORK 1\"), a Virginia Corporation with its principal place of business at 1501 Farm Credit Drive, Suite 1500, McLean, Virginia 22102-5004, and Payment Data Systems, Inc., the Affiliate Office (\"AFFILIATE\"), a Nevada Corporation with its principal place of business at 12500 San Pedro Suite 120 San Antonio, TX 78216.",
"Category": "Parties and Definitions"
},
{
"Text": "The term (\"Term\") of this Agreement shall be for one hundred eighty days (180) from the date set forth below unless Network 1 or Visa or MasterCard or Harris Bank doesn't approve Affiliate's ISO application, in which case, the Term will be 3 years. This Agreement will automatically renew for successive one-year terms unless terminated by either party by providing the other with 30 days written notice that this Agreement will not be renewed or Affiliate enters into a Processing agreement with Network 1 and an ISO Sponsorship agreement with Harris Bank in which case this Agreement will automatically terminate concurrent with the execution of such agreements.",
"Category": "Term and Renewal"
},
{
"Text": "Agreement may be terminated prior to the conclusion of the Term by giving written notice of termination: A. By either party as a result of default by the other party under this Agreement and failure to cure said default within thirty (30) days after notice of said default is given.",
"Category": "Termination Clause"
},
{
"Text": "Affiliate hereby agrees to indemnify and hold harmless Network 1, VISA, MasterCard and the Member Bank from and against any loss, cost or damage (including reasonable legal fees and court costs) incurred by Network 1, VISA, MasterCard and the Member Bank as a result of Affiliate's failure to comply with the terms of this Agreement, Affiliate's misrepresentation with respect to this Agreement or Affiliate's knowing or negligent misrepresentation with respect to Contractors.",
"Category": "Indemnification Clause"
},
{
"Text": "All disputes or claims hereunder shall be resolved by arbitration in McLean, Virginia, pursuant to the rules of the American Arbitration Association.",
"Category": "Arbitration/Dispute Resolution"
},
{
"Text": "This agreement may be assigned or delegated, in whole or in part, by NETWORK 1 without the prior written consent of the other party herein. This agreement may not be assigned or delegated by Affiliate without prior written consent from Network 1. Such consent shall not be unreasonably withheld.",
"Category": "Assignment"
}
]
================================================================================

[INFO] Searching Tavily for query: site:govinfo.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020
[INFO] Tavily results: {'query': 'site:govinfo.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.govinfo.gov/link/fr/89/38293', 'title': 'Federal Register/Vol. 89, No. 89/Tuesday, May 7, 2024/ ...', 'content': 'SUPPLEMENTARY INFORMATION. Compliance dates: Compliance with this rule is required no later than June 6, 2024, except for the following requirements: 1. 90', 'score': 0.100022875, 'raw_content': None}, {'url': 'https://www.govinfo.gov/content/pkg/FR-2021-02-12/pdf/2021-01499.pdf', 'title': 'Rules and Regulations', 'content': 'This section of the FEDERAL REGISTER contains regulatory documents having general applicability and legal effect, most of which are keyed to and codified in', 'score': 0.08688758, 'raw_content': None}, {'url': 'https://www.govinfo.gov/content/pkg/FR-2022-01-28/pdf/2022-01607.pdf', 'title': 'Rules and Regulations', 'content': 'This section of the FEDERAL REGISTER contains regulatory documents having general applicability and legal effect, most of which are keyed to and codified in', 'score': 0.08688758, 'raw_content': None}, {'url': 'https://www.govinfo.gov/content/pkg/FR-2021-02-23/pdf/2020-28473.pdf', 'title': 'Rules and Regulations', 'content': 'This includes examining the insured depository institution for compliance with laws and regulations, including affiliate transaction limits and capital', 'score': 0.0767739, 'raw_content': None}, {'url': 'https://www.govinfo.gov/content/pkg/STATUTE-92/pdf/STATUTE-92-Pg1824.pdf', 'title': 'Public Law 95-521 95th Congress An Act', 'content': '(6) If steps for assuring compliance with applicable laws and regulations are not taken by the date set under paragraph (3) by any other officer or employee the', 'score': 0.04646909, 'raw_content': None}, {'url': 'https://www.govinfo.gov/content/pkg/STATUTE-92/pdf/STATUTE-92-Pg607.pdf', 'title': 'Public Law 95-369 95th Congress An Act', 'content': 'As used in this subsection, the term "affiliate" shall mean Definitions, any company more than 5 per centum of whose voting shares is directly or indirectly', 'score': 0.043353733, 'raw_content': None}, {'url': 'https://www.govinfo.gov/content/pkg/STATUTE-65/pdf/STATUTE-65-Pg7-2.pdf', 'title': 'Public Law 8 Public Law 9 TITLE I—RENEGOTIATION OF ...', 'content': 'A provision inserted in a contract or subcontract, which recites in substance that the contract or subcontract shall be deemed to contain all the provisions', 'score': 0.030935358, 'raw_content': None}, {'url': 'https://www.govinfo.gov/content/pkg/FR-2025-01-08/pdf/2024-31486.pdf', 'title': 'Federal Register/Vol. 90, No. 5/Wednesday, January 8, ...', 'content': 'Executive Summary Executive Order 14117 of February 28, 2024, ‘‘Preventing Access to Americans’ Bulk Sensitive Personal Data and United States Government- Related Data by Countries of Concern’’ (‘‘the Order’’), directs the Attorney General to issue regulations that prohibit or otherwise restrict United States persons from engaging in any acquisition, holding, use, transfer, transportation, or exportation of, or dealing in, any property in which a foreign country or national thereof has any interest (‘‘transaction’’), where the transaction: involves United States Government-related data (‘‘government- related data’’) or bulk U.S. sensitive personal data, as defined by final rules implementing the Order; falls within a class of transactions that has been determined by the Attorney General to pose an unacceptable risk to the national security of the United States because it may enable access by countries of concern or covered persons to government-related data or Americans’ bulk U.S. sensitive personal data; and meets other criteria specified by the Order.1 VerDate Sep<11>2014 18:55 Jan 07, 2025 Jkt 265001 PO 00000 Frm 00002 Fmt 4701 Sfmt 4700 E:\\FR\\FM\\08JAR2.SGM 08JAR2 lotter on DSK11XQN23PROD with RULES2 1637 Federal Register / Vol. 90, No. 5 / Wednesday, January 8, 2025 / Rules and Regulations 2 89 FR 15780 (Mar. 5, 2024).', 'score': 0.023529684, 'raw_content': None}, {'url': 'https://www.govinfo.gov/content/pkg/FR-2024-01-18/pdf/2023-28629.pdf', 'title': 'Federal Register/Vol. 89, No. 12/Thursday, January 18, ...', 'content': "Insurance Corporation (FDIC) is amending its regulations governing use of the official FDIC sign and insured depository institutions' (IDIs).", 'score': 0.019009009, 'raw_content': None}, {'url': 'https://www.govinfo.gov/content/pkg/CRPT-119hrpt36/pdf/CRPT-119hrpt36.pdf', 'title': 'R E P O R T', 'content': '—A certification that a business concern qualifies as a small business concern of the exact size and status claimed by the business concern for purposes of', 'score': 0.018280081, 'raw_content': None}], 'response_time': 4.66}
[INFO] Searching Tavily for query: site:ecfr.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020
[INFO] Tavily results: {'query': 'site:ecfr.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.ecfr.gov/current/title-5/chapter-XVI/subchapter-B/part-2635', 'title': '5 CFR Part 2635 -- Standards of Ethical Conduct for ...', 'content': 'In addition to the regulations set forth in this part, employees must comply with any supplemental agency regulations issued by their employing agencies under', 'score': 0.085411474, 'raw_content': None}, {'url': 'https://www.ecfr.gov/current/title-13/chapter-I/part-121', 'title': '13 CFR Part 121 -- Small Business Size Regulations', 'content': '(1) Concerns and entities are affiliates of each other when one controls or has the power to control the other, or a third party or parties controls or has the', 'score': 0.07343685, 'raw_content': None}, {'url': 'https://www.ecfr.gov/current/title-49/subtitle-A/part-26', 'title': '49 CFR Part 26 -- Participation by Disadvantaged Business ...', 'content': '(a) A recipient must implement appropriate mechanisms to ensure compliance with the requirements in this part by all program participants (e.g., applying legal', 'score': 0.04039294, 'raw_content': None}], 'response_time': 4.81}
[INFO] Searching Tavily for query: site:law.cornell.edu "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020
[INFO] Tavily results: {'query': 'site:law.cornell.edu "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.law.cornell.edu/uscode/text/29/1002', 'title': '29 U.S. Code § 1002 - Definitions - Law.Cornell.Edu', 'content': '(D) Good faith compliance with law before guidance.—. An employer or pooled plan provider shall not be treated as failing to meet a requirement of guidance', 'score': 0.13933809, 'raw_content': None}, {'url': 'https://www.law.cornell.edu/uscode/text/15/632', 'title': '15 U.S. Code § 632 - Definitions - Law.Cornell.Edu', 'content': 'Each business certified as a small business concern under this chapter shall annually certify its small business size and, if appropriate, its small business', 'score': 0.08085701, 'raw_content': None}, {'url': 'https://www.law.cornell.edu/uscode/text/15/637', 'title': '15 U.S. Code § 637 - Additional powers - Cornell Law School', 'content': 'Limitations established by the Administration in its regulations and procedures restricting the award of contracts pursuant to this subsection to a limited', 'score': 0.07685699, 'raw_content': None}, {'url': 'https://www.law.cornell.edu/uscode/text/20/1094', 'title': '20 U.S. Code § 1094 - Program participation agreements', 'content': 'The agreement shall condition the initial and continuing eligibility of an institution to participate in a program upon compliance with the following', 'score': 0.061452325, 'raw_content': None}, {'url': 'https://www.law.cornell.edu/uscode/text/31/5318', 'title': '31 U.S. Code § 5318 - Compliance, exemptions, and ...', 'content': '(I). in carrying out clause (i), shall establish standards to ensure that streamlined reports relate to suspicious transactions relevant to potential violations', 'score': 0.050018918, 'raw_content': None}, {'url': 'https://www.law.cornell.edu/definitions/uscode.php?width=840&height=800&iframe=true&def_id=29-USC-1276085455-1337375897&term_occur=999&term_src=title:29:chapter:18:subchapter:I:subtitle:B:part:4:section:1108', 'title': 'Definition: direct compensation from 29 USC § 1108(b)(2)', 'content': '(cc) The term “affiliate”, with respect to a covered service provider, means an entity that directly or indirectly (through one or more intermediaries) controls', 'score': 0.043256637, 'raw_content': None}, {'url': 'https://www.law.cornell.edu/uscode/text/31/5336', 'title': '31 U.S. Code § 5336 - Beneficial ownership information ...', 'content': 'On and after the effective date of the regulations promulgated under subsection (b)(4), if the Secretary of the Treasury makes a determination, which may be based on information contained in the report required under section 6502(c) of the Anti-Money Laundering Act of 2020 or on any other information available to the Secretary, that an entity or class of entities described in subsection (a)(11)(B) has been involved in significant abuse relating to money laundering, the financing of terrorism, proliferation finance, serious tax fraud, or any other financial crime, not later than 90 days after the date on which the Secretary makes the determination, the Secretary shall submit to the Committee on Banking, Housing, and Urban Affairs of the Senate and the Committee on Financial Services of the House of Representatives a report that explains the reasons for the determination and any administrative or legislative recommendations to prevent such abuse.', 'score': 0.028829927, 'raw_content': None}, {'url': 'https://www.law.cornell.edu/uscode/text/15/648', 'title': '15 U.S. Code § 648 - Small business development center ...', 'content': 'No more than three members shall be from universities or their affiliates and six shall be from small businesses or associations representing small businesses.', 'score': 0.025841616, 'raw_content': None}, {'url': 'https://www.law.cornell.edu/cfr/text/13/128.402', 'title': '13 CFR § 128.402 - When may a joint venture submit an offer ...', 'content': '(B) The parties will perform the contract in compliance with the joint venture agreement and with the limitations on subcontracting requirements set forth in', 'score': 0.024914194, 'raw_content': None}, {'url': 'https://www.law.cornell.edu/regulations/pennsylvania/52-Pa-Code-SS-63-324', 'title': '52 Pa. Code § 63.324 - Commission approval of a general ...', 'content': '(a) General rule transactions. The following transactions of an applicant involving a change in conditions of service or rates that seek Commission approval', 'score': 0.009837972, 'raw_content': None}], 'response_time': 4.51}
[INFO] Searching Tavily for query: site:federalregister.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020
[INFO] Tavily results: {'query': 'site:federalregister.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://ecfr.federalregister.gov/current/title-2/part-200/section-200.1', 'title': '2 CFR 200.1 -- Definitions.', 'content': 'The Code of Federal Regulations (CFR) is the official legal print publication containing the codification of the general and permanent rules published in the', 'score': 0.06004818, 'raw_content': None}, {'url': 'https://ecfr.federalregister.gov/current/title-2/subtitle-A/chapter-II/part-200', 'title': '2 CFR Part 200 -- Uniform Administrative Requirements, ...', 'content': 'The Electronic Code of Federal Regulations (eCFR) is a continuously updated online version of the CFR. It is not an official legal edition of the CFR. Learn', 'score': 0.04924507, 'raw_content': None}, {'url': 'https://www.federalregister.gov/select-citation/2019/04/01/13-CFR-107', 'title': '13 CFR Part 107 -- Small Business Investment Companies', 'content': 'All Licensees must comply with all applicable regulations, accounting guidelines and valuation guidelines for Licensees.', 'score': 0.042202137, 'raw_content': None}, {'url': 'https://ecfr.federalregister.gov/current/title-13/part-121', 'title': '13 CFR Part 121 -- Small Business Size Regulations', 'content': "SBA's size standards define whether a business entity is small and, thus, eligible for Government programs and preferences reserved for “small business”", 'score': 0.027735045, 'raw_content': None}], 'response_time': 4.53}
[INFO] Searching Tavily for query: site:justice.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020
[INFO] Tavily results: {'query': 'site:justice.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.justice.gov/archives/opa/media/1341411/dl?inline', 'title': 'Obligations of foreign-based persons to comply with US ...', 'content': 'This Note highlights the applicability of U.S. sanctions and export control laws to persons and entities located abroad, as well as the enforcement mechanisms that are available for the U.S. government to hold non-U.S. persons accountable for violations of such laws, including criminal prosecution. In addition, under the EAR, certain foreign-produced items located outside of the United States that are produced using certain U.S.-controlled technology, software, or production equipment are subject to the EAR when exported from abroad, reexported, or transferred in-country to certain countries or parties on the Entity List. CRIMINAL ENFORCEMENT OF U.S. SANCTIONS AND EXPORT CONTROL LAWS AGAINST FOREIGN PERSONS AND ENTITIES As with any company participating in the global marketplace, foreign-based persons must ensure that they have robust compliance measures in place to avoid violating U.S. sanctions or export control laws.', 'score': 0.044138066, 'raw_content': None}, {'url': 'https://www.justice.gov/jm/jm-9-28000-principles-federal-prosecution-business-organizations', 'title': '9-28.000 - Principles of Federal Prosecution Of Business ...', 'content': "9-28.010Foundational Principles of Corporate Prosecution9-28.100Duties of Federal Prosecutors and Duties of Corporate Leaders9-28.200General Considerations of Corporate Liability9-28.210Focus on Individual Wrongdoers9-28.300Factors to Be Considered9-28.400Special Policy Concerns9-28.500Pervasiveness of Wrongdoing Within the Corporation9-28.600The Corporation's History of Misconduct9-28.700The Value of Cooperation9-28.710Attorney-Client and Work Product Protections9-28.720Cooperation: Disclosing the Relevant Facts9-28.730Obstructing the Investigation9-28.740Offering Cooperation: No Entitlement to Immunity9-28.750Oversight Concerning Demands for Waivers of Attorney-Client Privilege or Work Product Protection By Corporations Contrary to This Policy9-28.800Corporate Compliance Programs9-28.900Voluntary Self-Disclosures9-28.1000Restitution and Remediation9-28.1100Collateral Consequences9-28.1200Civil or Regulatory Alternatives9-28.1300Adequacy of Prosecution of Individuals9-28.1400Interests of the Victims and Others Significantly Harmed9-28.1500Selecting Charges9-28.1600Plea Agreements with Corporations9-28.1700Use of Independent Compliance Monitors in Corporate Resolution9-28.1710Approval of Determinations Concerning Monitors9-28.1720Selection of Monitor9-28.1740Continued Review and Scoping of Monitorships", 'score': 0.029093578, 'raw_content': None}, {'url': 'https://www.justice.gov/opa/media/1396356/dl', 'title': 'NSD Data Security Program - Compliance Guide - 04112025', 'content': "Such a risk assessment could assess coverage of the regulations against the company's current data holdings and vendor, employee, or investment agreements.", 'score': 0.022683308, 'raw_content': None}], 'response_time': 4.91}
[INFO] Searching Tavily for query: site:whitehouse.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020
[INFO] Tavily results: {'query': 'site:whitehouse.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [], 'response_time': 4.72}
{'messages': [HumanMessage(content='You are given a file path the document which you must preprocess to extract clauses. Once the clauses are extracted, fetch all relevant knowledge related to it. Based on the collected knowledge, you should check for compliance of these clauses. Explain the non-compliant clauses, suggest changes and summarize the results for the User. FILE PATH OF DOCUMENT: "./Original and Modified/modified_UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf" ",\n ', additional_kwargs={}, response_metadata={}, id='fe2bb3e4-24cc-4e10-ba7b-db4c831cee92'), AIMessage(content="Okay, I understand the problem. I need to analyze the legal document at the provided file path for discrepancies and compliance issues. Here's my plan:\n\n1. **Status Check:** I will first check the status of the Preprocessor, Knowledge, Compliance Checker, and Post-Processor agents to avoid redundant computations and ensure the workflow progresses correctly.\n2. **Preprocess Document:** If the Preprocessor agent hasn't completed its task, I will trigger it to preprocess the document, extract clauses, and classify them.\n3. **Retrieve Relevant Knowledge:** Once preprocessing is complete, I will trigger the Knowledge agent to retrieve relevant legal compliance knowledge from the web.\n4. **Check Clause Compliance:** After the Knowledge agent retrieves the necessary information, I will trigger the Compliance Checker agent to check the compliance of the clauses with legal regulations.\n5. **Summarize and Finalize Output:** Finally, upon clause revision, I will trigger the Post Processor agent to summarize the issues, suggest changes, and finalize the output for the user.\n\nNow, let's start by triggering the Preprocessor agent since this is the first step in the workflow.", additional_kwargs={'function_call': {'name': 'transfer_to_pre_processor_agent', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='Planner Agent', id='run-9ee0db2d-716f-4c7d-8e8d-5f20292ebb44-0', tool_calls=[{'name': 'transfer_to_pre_processor_agent', 'args': {}, 'id': '684cb1c1-3f73-4248-9e5e-6bbdb03b7bd5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 636, 'output_tokens': 248, 'total_tokens': 884, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Successfully transferred to Pre Processor Agent', name='transfer_to_pre_processor_agent', id='38bfb9e7-f45f-4ffc-b06d-680d47b93257', tool_call_id='684cb1c1-3f73-4248-9e5e-6bbdb03b7bd5'), AIMessage(content='Okay, I will now preprocess the document at the file path "./Original and Modified/modified_UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf" to extract the full text, classify the document, identify named entities, and extract key clauses. After processing, I will store all the extracted information in a JSON object.', additional_kwargs={'function_call': {'name': 'preprocess_document_tool', 'arguments': '{"file_path": "./Original and Modified/modified_UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='Pre Processor Agent', id='run-36cacf01-6840-4759-ab1e-0ee92c07813a-0', tool_calls=[{'name': 'preprocess_document_tool', 'args': {'file_path': './Original and Modified/modified_UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf'}, 'id': 'e222374b-d49b-4d91-91ac-a5164c38f64f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 530, 'output_tokens': 164, 'total_tokens': 694, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='{"Document Title": "Based on the provided text, the title of the document is:\\n\\n**NETWORK 1 FINANCIAL CORPORATION AFFILIATE OFFICE AGREEMENT**", "Document Class": "Legal Agreement - Affiliate Office Agreement"}', name='preprocess_document_tool', id='a08930b8-315e-404f-97db-b7940f2687ae', tool_call_id='e222374b-d49b-4d91-91ac-a5164c38f64f'), AIMessage(content='OK. I have preprocessed the document. Now I need to signal to the Planner Agent that I am done.', additional_kwargs={'function_call': {'name': 'transfer_to_planner_agent', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='Pre Processor Agent', id='run-b43ea8da-97e0-4b56-927f-68d2f7f1de94-0', tool_calls=[{'name': 'transfer_to_planner_agent', 'args': {}, 'id': 'b1b9db6f-1131-4ab1-8854-0a54b27441c0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 633, 'output_tokens': 31, 'total_tokens': 664, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Successfully transferred to Planner Agent', name='transfer_to_planner_agent', id='7c9eea30-198c-4c1b-91d8-498f4559affb', tool_call_id='b1b9db6f-1131-4ab1-8854-0a54b27441c0'), AIMessage(content='Okay, the Pre Processor Agent has completed its task. Now, I will trigger the Knowledge Agent to retrieve relevant legal compliance knowledge from the web.', additional_kwargs={'function_call': {'name': 'transfer_to_knowledge_agent', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='Planner Agent', id='run-0e9e0129-4ab4-476b-90a1-f6ded49ead69-0', tool_calls=[{'name': 'transfer_to_knowledge_agent', 'args': {}, 'id': '5395a90d-9e37-4a9a-a7d1-df180858eceb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 784, 'output_tokens': 37, 'total_tokens': 821, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Successfully transferred to Knowledge Agent', name='transfer_to_knowledge_agent', id='3478ee58-e761-435e-853e-2c8f71a91004', tool_call_id='5395a90d-9e37-4a9a-a7d1-df180858eceb'), AIMessage(content='Okay, I will start by retrieving knowledge related to the affiliate office agreement.', additional_kwargs={'function_call': {'name': 'retrieve_web_knowledge_tool', 'arguments': '{"query": "site:govinfo.gov \\"Affiliate Office Agreement\\" AND \\"Compliance\\" AND (\\"Regulations\\" OR \\"Statutes\\") after:2020"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='Knowledge Agent', id='run-e9f292b2-a11f-4093-9981-87b58b81ab4b-0', tool_calls=[{'name': 'retrieve_web_knowledge_tool', 'args': {'query': 'site:govinfo.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020'}, 'id': 'a2497ca0-9cac-4d65-aa4b-2d1d4a9d681c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 670, 'output_tokens': 55, 'total_tokens': 725, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Error: UnboundLocalError("cannot access local variable \'url\' where it is not associated with a value")\n Please fix your mistakes.', name='retrieve_web_knowledge_tool', id='85f0eef0-2477-45d9-a06b-3e8fe61563f1', tool_call_id='a2497ca0-9cac-4d65-aa4b-2d1d4a9d681c', status='error'), AIMessage(content='I encountered an error while trying to retrieve information using the `retrieve_web_knowledge_tool`. I will try a different query.', additional_kwargs={'function_call': {'name': 'retrieve_web_knowledge_tool', 'arguments': '{"query": "site:ecfr.gov \\"Affiliate Office Agreement\\" AND \\"Compliance\\" AND (\\"Regulations\\" OR \\"Statutes\\") after:2020"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='Knowledge Agent', id='run-57db3701-f697-4f75-9a15-e4e72bb00a6a-0', tool_calls=[{'name': 'retrieve_web_knowledge_tool', 'args': {'query': 'site:ecfr.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020'}, 'id': 'af2f5eb7-2b20-4fb5-a645-694dfc15e961', 'type': 'tool_call'}], usage_metadata={'input_tokens': 746, 'output_tokens': 66, 'total_tokens': 812, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Error: UnboundLocalError("cannot access local variable \'url\' where it is not associated with a value")\n Please fix your mistakes.', name='retrieve_web_knowledge_tool', id='5a7deb63-a488-41f7-b5ca-b1c750b5d389', tool_call_id='af2f5eb7-2b20-4fb5-a645-694dfc15e961', status='error'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'retrieve_web_knowledge_tool', 'arguments': '{"query": "site:law.cornell.edu \\"Affiliate Office Agreement\\" AND \\"Compliance\\" AND (\\"Regulations\\" OR \\"Statutes\\") after:2020"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='Knowledge Agent', id='run-fd44c103-7784-45f7-8773-b8eb61b03e16-0', tool_calls=[{'name': 'retrieve_web_knowledge_tool', 'args': {'query': 'site:law.cornell.edu "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020'}, 'id': '294afca2-cfd4-4cd5-8aaa-f62b86e8e9c0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 821, 'output_tokens': 40, 'total_tokens': 861, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Error: UnboundLocalError("cannot access local variable \'url\' where it is not associated with a value")\n Please fix your mistakes.', name='retrieve_web_knowledge_tool', id='3d76bd38-3300-438f-9d8c-138197d820a3', tool_call_id='294afca2-cfd4-4cd5-8aaa-f62b86e8e9c0', status='error'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'retrieve_web_knowledge_tool', 'arguments': '{"query": "site:federalregister.gov \\"Affiliate Office Agreement\\" AND \\"Compliance\\" AND (\\"Regulations\\" OR \\"Statutes\\") after:2020"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='Knowledge Agent', id='run-c345954d-b24b-4b89-a383-7bc29d0a8fcb-0', tool_calls=[{'name': 'retrieve_web_knowledge_tool', 'args': {'query': 'site:federalregister.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020'}, 'id': 'e39f6b2f-d698-43fb-8c7b-8c2021af5e9e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 898, 'output_tokens': 38, 'total_tokens': 936, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Error: UnboundLocalError("cannot access local variable \'url\' where it is not associated with a value")\n Please fix your mistakes.', name='retrieve_web_knowledge_tool', id='ea374ca7-531b-4291-b75e-a7389dd6e24c', tool_call_id='e39f6b2f-d698-43fb-8c7b-8c2021af5e9e', status='error'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'retrieve_web_knowledge_tool', 'arguments': '{"query": "site:justice.gov \\"Affiliate Office Agreement\\" AND \\"Compliance\\" AND (\\"Regulations\\" OR \\"Statutes\\") after:2020"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='Knowledge Agent', id='run-b74c8cd0-3819-42b9-82c4-d376f30d0bbb-0', tool_calls=[{'name': 'retrieve_web_knowledge_tool', 'args': {'query': 'site:justice.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020'}, 'id': '5a0e87d8-1aa4-4c4e-9c66-78ebba2ca31f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 973, 'output_tokens': 37, 'total_tokens': 1010, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Error: UnboundLocalError("cannot access local variable \'url\' where it is not associated with a value")\n Please fix your mistakes.', name='retrieve_web_knowledge_tool', id='b7f2485f-e2b4-4d3b-ad8a-38d41d8007e2', tool_call_id='5a0e87d8-1aa4-4c4e-9c66-78ebba2ca31f', status='error'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'retrieve_web_knowledge_tool', 'arguments': '{"query": "site:whitehouse.gov \\"Affiliate Office Agreement\\" AND \\"Compliance\\" AND (\\"Regulations\\" OR \\"Statutes\\") after:2020"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='Knowledge Agent', id='run-1bf7c86b-f785-4727-946f-b8ec50753d1b-0', tool_calls=[{'name': 'retrieve_web_knowledge_tool', 'args': {'query': 'site:whitehouse.gov "Affiliate Office Agreement" AND "Compliance" AND ("Regulations" OR "Statutes") after:2020'}, 'id': 'e56a5177-42ff-4a55-a311-4999d4ee224c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1047, 'output_tokens': 38, 'total_tokens': 1085, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Error: UnboundLocalError("cannot access local variable \'url\' where it is not associated with a value")\n Please fix your mistakes.', name='retrieve_web_knowledge_tool', id='c1e8cfd3-a316-4400-8886-b21207c779dd', tool_call_id='e56a5177-42ff-4a55-a311-4999d4ee224c', status='error'), AIMessage(content='I was unable to retrieve relevant information regarding the compliance of "Affiliate Office Agreements" from the specified government sources. I will now inform the Planner Agent that the knowledge retrieval is completed.', additional_kwargs={'function_call': {'name': 'transfer_to_planner_agent', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='Knowledge Agent', id='run-4cee2fdc-0f7d-4198-beb0-478198e4aac8-0', tool_calls=[{'name': 'transfer_to_planner_agent', 'args': {}, 'id': '6ed263a4-1d10-4bcd-8a15-8d01e9d00dd6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1122, 'output_tokens': 44, 'total_tokens': 1166, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Successfully transferred to Planner Agent', name='transfer_to_planner_agent', id='05deddfc-9ec4-4c2e-bbe0-a03cd084a7c2', tool_call_id='6ed263a4-1d10-4bcd-8a15-8d01e9d00dd6'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'transfer_to_compliance_checker_agent', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='Planner Agent', id='run-bea976d3-3286-4e80-8dfd-87beffd5015e-0', tool_calls=[{'name': 'transfer_to_compliance_checker_agent', 'args': {}, 'id': '2f4f4f81-8b96-4841-9e20-d50c1c8983a2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1276, 'output_tokens': 9, 'total_tokens': 1285, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Successfully transferred to Compliance Checker Agent', name='transfer_to_compliance_checker_agent', id='e433e9b5-dc91-45ab-9360-da94b65169ba', tool_call_id='2f4f4f81-8b96-4841-9e20-d50c1c8983a2'), AIMessage(content='I am unable to proceed with the compliance check because I need the clauses from the document. I have already transferred the request to the Pre Processor Agent to extract the clauses. After the clauses are extracted, I will need to fetch relevant knowledge related to it. Based on the collected knowledge, I should check for compliance of these clauses.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='Compliance Checker Agent', id='run-af27177e-29b6-4882-a505-7ced15d679ac-0', usage_metadata={'input_tokens': 1397, 'output_tokens': 67, 'total_tokens': 1464, 'input_token_details': {'cache_read': 0}})], 'active_agent': 'Compliance Checker Agent'}
